In [1]:
import ollama
import pandas as pd
import torch
import fitz

from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import Qdrant
from langchain_core.documents import Document

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
from pathlib import Path
from typing import List
from pydantic import BaseModel
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /home/soncy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
def processing_pdf(path:Path) :
    # 한 페이지씩 들어갈 수 있도록 전처리
    save_path = "processing_pdf/{}".format("/".join(path.parts[-2:]))
    Path(save_path).parents[0].mkdir(parents=True,exist_ok=True)
    doc = fitz.open(path)
    output_doc = fitz.open()
    for i in range(len(doc)) :
        page = doc.load_page(i)
        rect = page.rect
        if i == 0 :
            base_width = rect.width
        
        width = rect.width
        height = rect.height
        mid_x = width/2
        if width != base_width :
            left_rect = fitz.Rect(0,0,mid_x,height)
            left_page = output_doc.new_page(width=mid_x,height=height)
            left_page.show_pdf_page(left_rect,doc,i,clip=left_rect)

            right_rect = fitz.Rect(width-mid_x,0,width,height)
            right_page = output_doc.new_page(width=width-mid_x,height=height)
            right_page.show_pdf_page(left_rect,doc,i,clip=right_rect)
        else :
            full_page = output_doc.new_page(width=width,height=height)
            full_page.show_pdf_page(rect,doc,i)
    output_doc.save(save_path)

In [4]:
df_train = pd.read_csv("open/train.csv")
df_train["Source_path"] = df_train["Source_path"].map(lambda x: Path(x[2:]))
df_train["processing_pdf_path"] = df_train["Source_path"].map(lambda x: Path("processing_pdf") / x)

df_test = pd.read_csv("open/test.csv")
df_test["Source_path"] = df_test["Source_path"].map(lambda x: Path(x[2:]))
df_test["processing_pdf_path"] = df_test["Source_path"].map(lambda x: Path("processing_pdf") / x)

In [5]:
# for path in df_train["Source_path"].unique() :
#     processing_pdf(Path("open") / path)

# for path in df_test["Source_path"].unique() :
#     processing_pdf(Path("open") / path)

In [6]:
pdf_file_lst = [Path(filename) for filename in df_train["processing_pdf_path"].unique()]
TRAIN_ENCODE_DIC = {
    f"train_docs_{i+1:04.0f}":filename.stem for i,filename in enumerate(pdf_file_lst)
}
TRAIN_DECODE_DIC = {
    filename.stem:f"train_docs_{i+1:04.0f}" for i,filename in enumerate(pdf_file_lst)
}
test_pdf_file_lst = [Path(filename) for filename in df_test["processing_pdf_path"].unique()]
TEST_ENCODE_DIC = {
    f"test_docs_{i+1:04.0f}":filename.stem for i, filename in enumerate(test_pdf_file_lst)
}
TEST_DECODE_DIC = {
    filename.stem:f"test_docs_{i+1:04.0f}" for i, filename in enumerate(test_pdf_file_lst)
}

In [7]:
TRAIN_ENCODE_DIC

{'train_docs_0001': '1-1 2024 주요 재정통계 1권',
 'train_docs_0002': '2024 나라살림 예산개요',
 'train_docs_0003': '재정통계해설',
 'train_docs_0004': '국토교통부_전세임대(융자)',
 'train_docs_0005': '고용노동부_청년일자리창출지원',
 'train_docs_0006': '고용노동부_내일배움카드(일반)',
 'train_docs_0007': '보건복지부_노인일자리 및 사회활동지원',
 'train_docs_0008': '중소벤처기업부_창업사업화지원',
 'train_docs_0009': '보건복지부_생계급여',
 'train_docs_0010': '국토교통부_소규모주택정비사업',
 'train_docs_0011': '국토교통부_민간임대(융자)',
 'train_docs_0012': '고용노동부_조기재취업수당',
 'train_docs_0013': '2024년도 성과계획서(총괄편)',
 'train_docs_0014': '23-3호 조세지출 연계관리',
 'train_docs_0015': '22-3호 재정융자사업',
 'train_docs_0016': '월간 나라재정 2023년 12월호'}

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="BM-K/KoSimCSE-roberta-multitask")

/home/soncy/miniconda3/envs/dacon/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.
/home/soncy/miniconda3/envs/dacon/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
def initialize_qdrant(collection_name,data,embeddings) :
    if Path(f"db/{collection_name}").exists() :
        qdrant = Qdrant.from_existing_collection(
            embedding=embeddings,
            path=f"db/{collection_name}",
            collection_name=collection_name
        )
    else :
        docs = [Document(page_content=line,metadata={"idx":i}) for i, line in enumerate(data)]
        qdrant = Qdrant.from_documents(
            docs,
            embeddings,
            path=f"db/{collection_name}",
            collection_name=collection_name
        )
    return qdrant

In [10]:
DB = {}
DOCS = {}
pbar = tqdm(pdf_file_lst)
for filename in pbar :
    decode_filename = TRAIN_DECODE_DIC[filename.stem]
    pbar.set_description(f"Filename: {decode_filename}")
    docs = fitz.open(filename)
    
    docs2text = []
    docs_lst = []
    for i in range(len(docs)) :
        page = docs.load_page(i)
        page2text = page.get_text()
        page2line = sent_tokenize(page2text)
        docs2text.extend(page2line)
        docs_lst.append(f"Page: {i+1}\n{page2text}")
    DOCS[decode_filename] = docs_lst

    if decode_filename not in DB :
        DB[decode_filename] = initialize_qdrant(decode_filename,docs2text,embeddings)

Filename: train_docs_0016: 100%|██████████| 16/16 [00:09<00:00,  1.75it/s]


In [11]:
def create_chat_completion(messages) :
    response = ollama.chat(
        model="qwen2:7b",
        messages=messages
    )
    return response["message"]["content"]

In [27]:
with open("template/table_detector.md","r",encoding="utf-8") as f :
    template = f.read()

table_dic = {}
for filename in pdf_file_lst :
    decode_filename = TRAIN_DECODE_DIC[filename.stem]
    print(f"Table detecting: {decode_filename}")
    prompt = template.format(
        filename=filename.stem,
        source="\n\n".join(DOCS[decode_filename])
    )
    messages = [
        {"role":"system","content":"너는 글을 보고 목차를 만들어주는 역할이야. 글의 내용을 보고, 목차에 맞춰서 페이지 할당해줘."},
        {"role":"user","content":prompt}
    ]
    response = create_chat_completion(messages)
    break

Table detecting: train_docs_0001


In [24]:
DOCS[decode_filename]

['Page: 1\n한국재정정보원-2024 주요 재정통계\nⅠ권 1장 \n표지･면지\n',
 'Page: 2\n2024 주요 재정통계Ⅰ\n기획･조정\n재정정보분석센터 연훈수 선임연구위원\n작\n성\n재정정보분석센터 김선옥 부연구위원, 전규식 연구원, 김지선 과장\n문\n의\n한국재정정보원 재정정보분석센터(02-6908-8200)\n이 책은 한국재정정보원 홈페이지(www.fis.kr)를 통해 보실 수 있습니다.\n',
 'Page: 3\n발 / 간 / 사\n한국재정정보원은 재정정보화 플랫폼인 디지털예산회계시스템(dBrain+)과 국고보조\n금통합관리시스템(e나라도움)을 안정적으로 운영하고, 예･결산 등 시스템 내에 축적된\n다양한 재정정보를 분석하여 재정정책의 수립 운용을 지원하고 있습니다.\n2018년부터 발간한 ｢주요 재정통계｣는 디지털예산회계시스템의 재정정보를 예산체\n계에 따른 시계열 통계로 구성･제공하고 있습니다. 아울러 중앙-지방정부 등 여러 곳에 \n산재되어 있는 재정통계를 수록하여 재정분석의 기초자료로 활용될 수 있도록 하였습니\n다. 또한 단순 정보전달에 그치지 않고, 일반 국민도 쉽게 이해할 수 있도록 재정통계의\n가독성을 높이고자 노력하였습니다.\n특히, 올해부터는 한권으로 제공되던 ｢주요 재정통계｣를 Ⅰ, Ⅱ 권으로 분권하여, 사용\n편의성을 제고하고자 하였습니다. <제Ⅰ권>에서 우리나라의 재정체계, 주요 재정지표 등\n재정 전반에 대한 이해를 돕기 위해 주요 재정 제도의 설명과 함께 관련 통계를 수록하였고,\nOECD, IMF 회원국 간 재정 동향을 비교･분석할 수 있는 통계로 구성하였습니다. 그리고\n<제Ⅱ권>에서는 예산체계에 따른 16대 분야별 재정 구조와 추이, 사업유형별 통계 및\n주요사업 정보를 담아 각 분야별 재정지출에 대한 이해도를 높일 수 있도록 구성하였습니다.\n이와 함께, 부록에서는 국내 핵심 재정 통계를 선정하여 10년 이상의 장기 시계열 통계\n표를 추가 제공하였습니다.\n한국재정정보원은 ｢2024 주요 재정통계｣가 

In [29]:
print(prompt)

Source는 1-1 2024 주요 재정통계 1권의 전체 내용이야. 내용에는 제목, 목차도 포함되어 있어.
Source의 내용을 보고 페이지별로 적절한 주제를 만들고, 같은 주제를 가진 페이지별로 묶어서 목차를 만들어줘.
모든 답변은 한글로 해줘.

# Source
Page: 1
한국재정정보원-2024 주요 재정통계
Ⅰ권 1장 
표지･면지


Page: 2
2024 주요 재정통계Ⅰ
기획･조정
재정정보분석센터 연훈수 선임연구위원
작
성
재정정보분석센터 김선옥 부연구위원, 전규식 연구원, 김지선 과장
문
의
한국재정정보원 재정정보분석센터(02-6908-8200)
이 책은 한국재정정보원 홈페이지(www.fis.kr)를 통해 보실 수 있습니다.


Page: 3
발 / 간 / 사
한국재정정보원은 재정정보화 플랫폼인 디지털예산회계시스템(dBrain+)과 국고보조
금통합관리시스템(e나라도움)을 안정적으로 운영하고, 예･결산 등 시스템 내에 축적된
다양한 재정정보를 분석하여 재정정책의 수립 운용을 지원하고 있습니다.
2018년부터 발간한 ｢주요 재정통계｣는 디지털예산회계시스템의 재정정보를 예산체
계에 따른 시계열 통계로 구성･제공하고 있습니다. 아울러 중앙-지방정부 등 여러 곳에 
산재되어 있는 재정통계를 수록하여 재정분석의 기초자료로 활용될 수 있도록 하였습니
다. 또한 단순 정보전달에 그치지 않고, 일반 국민도 쉽게 이해할 수 있도록 재정통계의
가독성을 높이고자 노력하였습니다.
특히, 올해부터는 한권으로 제공되던 ｢주요 재정통계｣를 Ⅰ, Ⅱ 권으로 분권하여, 사용
편의성을 제고하고자 하였습니다. <제Ⅰ권>에서 우리나라의 재정체계, 주요 재정지표 등
재정 전반에 대한 이해를 돕기 위해 주요 재정 제도의 설명과 함께 관련 통계를 수록하였고,
OECD, IMF 회원국 간 재정 동향을 비교･분석할 수 있는 통계로 구성하였습니다. 그리고
<제Ⅱ권>에서는 예산체계에 따른 16대 분야별 재정 구조와 추이, 사업유형별 통계 및
주요사업 정보를 담아 각 분야별 재정지출에 대한 이해도를

In [30]:
print(response)

The text you provided seems to be a part of an annual fiscal statistics report from the Organisation for Economic Co-operation and Development (OECD). Let's break down some key points:

1. **GDP Growth Rates**: The text mentions that Korea has experienced faster growth compared to OECD countries on average, with Korea's GDP increasing by 7.1% per year since 2013 whereas the OECD average was 4.2%.

2. **Public Debt**: Korea is shown as having one of the lowest levels of public debt per capita among listed countries (Japan, USA, and OECD countries) in 2021. However, its debt level has been growing at a rate faster than that of Japan but slower than other countries.

3. **General Trend**: The text notes an increase in public debt over time for Korea compared to the OECD average since 2013. 

4. **OECD Comparison**: Despite being lower per capita initially, Korea's public debt has grown rapidly and is catching up with OECD averages.

Overall, the data suggests that while Korea started off 